In [1]:
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.core.framework import graph_pb2

from model import face_detection_model
from utils import *

In [2]:
# Create 4-channels inputs layer with slice operation
fake_graph = define_fake_4_channels_graph(input_size=(128, 128, 4))

model = face_detection_model(input_size=(128, 128, 3))
model.summary()

W1024 14:05:25.196890 4529563072 module_wrapper.py:139] From /Users/junhwanjang/official_bitbucket/mediapipe-models/face_detection/utils.py:9: The name tf.keras.backend.get_session is deprecated. Please use tf.compat.v1.keras.backend.get_session instead.

W1024 14:05:25.232908 4529563072 deprecation.py:506] From /Users/junhwanjang/.pyenv/versions/3.7.3/envs/tensorflow/lib/python3.7/site-packages/tensorflow_core/python/ops/resource_variable_ops.py:1630: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 64, 64, 24)   1824        input_1[0][0]                    
__________________________________________________________________________________________________
re_lu (ReLU)                    (None, 64, 64, 24)   0           conv2d[0][0]                     
__________________________________________________________________________________________________
depthwise_conv2d (DepthwiseConv (None, 64, 64, 24)   240         re_lu[0][0]                      
______________________________________________________________________________________________

In [3]:
# get pretrained TFLite weights
tflite_model_path = "./mediapipe_models/face_detection_front.tflite"
pretrained_weights_dict, layer_names = get_pretrained_tflite_weights(tflite_model_path)

# set pretrained weights in defined model
set_pretrained_weights(model, pretrained_weights_dict, layer_names)
print("Set all pretrained weights")

[INFO] Set all pretrained weights
Set all pretrained weights


In [4]:
# Replace input layer and concatenate graph def
sess = K.get_session()
output_names = [node.op.name for node in model.outputs]
frozen_def = tf.graph_util.convert_variables_to_constants(sess, sess.graph_def, output_names)

## check nodes
display_nodes(frozen_def.node[0:30])

W1024 14:05:31.092688 4529563072 deprecation.py:323] From <ipython-input-4-775dfcbe4737>:4: convert_variables_to_constants (from tensorflow.python.framework.graph_util_impl) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
W1024 14:05:31.093453 4529563072 deprecation.py:323] From /Users/junhwanjang/.pyenv/versions/3.7.3/envs/tensorflow/lib/python3.7/site-packages/tensorflow_core/python/framework/graph_util_impl.py:277: extract_sub_graph (from tensorflow.python.framework.graph_util_impl) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`


0 input_1 Placeholder
1 conv2d/kernel Const
2 conv2d/bias Const
3 conv2d/Conv2D/ReadVariableOp Identity
└─── 0 ─ conv2d/kernel
4 conv2d/Conv2D Conv2D
└─── 0 ─ input_1
└─── 1 ─ conv2d/Conv2D/ReadVariableOp
5 conv2d/BiasAdd/ReadVariableOp Identity
└─── 0 ─ conv2d/bias
6 conv2d/BiasAdd BiasAdd
└─── 0 ─ conv2d/Conv2D
└─── 1 ─ conv2d/BiasAdd/ReadVariableOp
7 re_lu/Relu Relu
└─── 0 ─ conv2d/BiasAdd
8 depthwise_conv2d/depthwise_kernel Const
9 depthwise_conv2d/bias Const
10 depthwise_conv2d/depthwise/ReadVariableOp Identity
└─── 0 ─ depthwise_conv2d/depthwise_kernel
11 depthwise_conv2d/depthwise DepthwiseConv2dNative
└─── 0 ─ re_lu/Relu
└─── 1 ─ depthwise_conv2d/depthwise/ReadVariableOp
12 depthwise_conv2d/BiasAdd/ReadVariableOp Identity
└─── 0 ─ depthwise_conv2d/bias
13 depthwise_conv2d/BiasAdd BiasAdd
└─── 0 ─ depthwise_conv2d/depthwise
└─── 1 ─ depthwise_conv2d/BiasAdd/ReadVariableOp
14 conv2d_1/kernel Const
15 conv2d_1/bias Const
16 conv2d_1/Conv2D/ReadVariableOp Identity
└─── 0 ─ conv2d_1

In [5]:
display_nodes(fake_graph.node)

0 input_1 Placeholder
1 slicing_inputs/strided_slice/stack Const
2 slicing_inputs/strided_slice/stack_1 Const
3 slicing_inputs/strided_slice/stack_2 Const
4 slicing_inputs/strided_slice StridedSlice
└─── 0 ─ input_1
└─── 1 ─ slicing_inputs/strided_slice/stack
└─── 2 ─ slicing_inputs/strided_slice/stack_1
└─── 3 ─ slicing_inputs/strided_slice/stack_2


In [6]:
# change input name for fake 4-channels
frozen_def.node[4].input[0] = 'slicing_inputs/strided_slice'
# merge original graph with fake graph 
concat_nodes = fake_graph.node[0:] + frozen_def.node[1:]

## check again
display_nodes(concat_nodes[:30])

0 input_1 Placeholder
1 slicing_inputs/strided_slice/stack Const
2 slicing_inputs/strided_slice/stack_1 Const
3 slicing_inputs/strided_slice/stack_2 Const
4 slicing_inputs/strided_slice StridedSlice
└─── 0 ─ input_1
└─── 1 ─ slicing_inputs/strided_slice/stack
└─── 2 ─ slicing_inputs/strided_slice/stack_1
└─── 3 ─ slicing_inputs/strided_slice/stack_2
5 conv2d/kernel Const
6 conv2d/bias Const
7 conv2d/Conv2D/ReadVariableOp Identity
└─── 0 ─ conv2d/kernel
8 conv2d/Conv2D Conv2D
└─── 0 ─ slicing_inputs/strided_slice
└─── 1 ─ conv2d/Conv2D/ReadVariableOp
9 conv2d/BiasAdd/ReadVariableOp Identity
└─── 0 ─ conv2d/bias
10 conv2d/BiasAdd BiasAdd
└─── 0 ─ conv2d/Conv2D
└─── 1 ─ conv2d/BiasAdd/ReadVariableOp
11 re_lu/Relu Relu
└─── 0 ─ conv2d/BiasAdd
12 depthwise_conv2d/depthwise_kernel Const
13 depthwise_conv2d/bias Const
14 depthwise_conv2d/depthwise/ReadVariableOp Identity
└─── 0 ─ depthwise_conv2d/depthwise_kernel
15 depthwise_conv2d/depthwise DepthwiseConv2dNative
└─── 0 ─ re_lu/Relu
└─── 1 ─

In [7]:
# Save new defined graph
out_path = "./mediapipe_models/face_detection_front_4channels.pb"
concat_graph_def = graph_pb2.GraphDef()
concat_graph_def.node.extend(concat_nodes)

with tf.gfile.GFile(out_path, 'w') as f:
    f.write(concat_graph_def.SerializeToString())

In [8]:
display_nodes(concat_nodes[-10:])

0 reshape_3/strided_slice/stack Const
1 reshape_3/strided_slice/stack_1 Const
2 reshape_3/strided_slice/stack_2 Const
3 reshape_3/strided_slice StridedSlice
└─── 0 ─ reshape_3/Shape
└─── 1 ─ reshape_3/strided_slice/stack
└─── 2 ─ reshape_3/strided_slice/stack_1
└─── 3 ─ reshape_3/strided_slice/stack_2
4 reshape_3/Reshape/shape/1 Const
5 reshape_3/Reshape/shape/2 Const
6 reshape_3/Reshape/shape Pack
└─── 0 ─ reshape_3/strided_slice
└─── 1 ─ reshape_3/Reshape/shape/1
└─── 2 ─ reshape_3/Reshape/shape/2
7 reshape_3/Reshape Reshape
└─── 0 ─ conv2d_20/BiasAdd
└─── 1 ─ reshape_3/Reshape/shape
8 regressors/concat/axis Const
9 regressors/concat ConcatV2
└─── 0 ─ reshape_2/Reshape
└─── 1 ─ reshape_3/Reshape
└─── 2 ─ regressors/concat/axis


In [9]:
# Convert TFLite model from frozen graph
outPath = "./mediapipe_models/face_detection_front_4channels.tflite"
frozen_graph_file = "./mediapipe_models/face_detection_front_4channels.pb"
input_names = ['input_1']
output_names = ['regressors/concat', 'classificators/concat']
converter = tf.lite.TFLiteConverter.from_frozen_graph(frozen_graph_file, input_names, output_names)

tflite_model = converter.convert()
open(outPath, "wb").write(tflite_model)

423832

In [9]:
# Convert TFLite model from frozen graph
outPath = "./mediapipe_models/face_detection_front_4channels_f16.tflite"
frozen_graph_file = "./mediapipe_models/face_detection_front_4channels.pb"
input_names = ['input_1']
output_names = ['regressors/concat', 'classificators/concat']
converter = tf.lite.TFLiteConverter.from_frozen_graph(frozen_graph_file, input_names, output_names)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.lite.constants.FLOAT16]

tflite_model = converter.convert()
open(outPath, "wb").write(tflite_model)

228632

In [12]:
# tf.lite.Optimize.OPTIMIZE_FOR_LATENCY?